In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [3]:
!pip install lightgbm
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
pd.options.mode.chained_assignment = None
from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score, f1_score,roc_auc_score
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
import pickle
import torch
torch.manual_seed(3)

In [5]:
#data frame with the filtered topic labels and features that characterize each data entry:

file = open("gsdmm_topics.pkl",'rb')
df = pickle.load(file)
file.close()


In [6]:
#Tweet count for all the users considered
df['engaging_user_id'].value_counts()

72C2BE10D3673447A6D596E12C0523DA    502
3396F36ADFF2A01A34C0CB3486CABFEE    388
5049F4AC51D668D064214377EB38A8D3    320
3FC1A3B3B9C8D1BD6673C3B5B65A6E91    308
F1D8FA7C91B5EE653330E80D48C77AD1    297
                                   ... 
72299BF202244855FD96178C95FEDBF2     21
88B6B62A1752C6547E0EE0DF34AB2DD7     21
05E4628760701D385318FE2A7BB1D952     21
AA2B9CC4D183BF7701A2F75239450AE4     21
DC5D486BE66A1C3C39B75FF2ECFE8A55     21
Name: engaging_user_id, Length: 107017, dtype: int64

In [7]:
#Convert date to yyyy-mm-dd hh:mm:ss format
import datetime
df['tweet_timestamp'] = df["tweet_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x)) 

In [8]:
t1 = df[(df["tweet_timestamp"] >= '2020-02-05 18:00:00') & (df["tweet_timestamp"] < '2020-02-06 06:00:00')]
t2 = df[(df["tweet_timestamp"] >= '2020-02-06 06:00:00') & (df["tweet_timestamp"] < '2020-02-06 18:00:00')]
t3 = df[(df["tweet_timestamp"] >= '2020-02-06 18:00:00') & (df["tweet_timestamp"] < '2020-02-07 06:00:00')]
t4 = df[(df["tweet_timestamp"] >= '2020-02-07 06:00:00') & (df["tweet_timestamp"] < '2020-02-07 18:00:00')]
t5 = df[(df["tweet_timestamp"] >= '2020-02-07 18:00:00') & (df["tweet_timestamp"] < '2020-02-08 06:00:00')]
t6 = df[(df["tweet_timestamp"] >= '2020-02-08 06:00:00') & (df["tweet_timestamp"] < '2020-02-08 18:00:00')]
t7 = df[(df["tweet_timestamp"] >= '2020-02-08 18:00:00') & (df["tweet_timestamp"] < '2020-02-09 06:00:00')]
t8 = df[(df["tweet_timestamp"] >= '2020-02-09 06:00:00') & (df["tweet_timestamp"] < '2020-02-09 18:00:00')]
t9 = df[(df["tweet_timestamp"] >= '2020-02-09 18:00:00') & (df["tweet_timestamp"] < '2020-02-10 06:00:00')]
t10 = df[(df["tweet_timestamp"] >= '2020-02-10 06:00:00') & (df["tweet_timestamp"] < '2020-02-10 18:00:00')]
t11 = df[(df["tweet_timestamp"] >= '2020-02-10 18:00:00') & (df["tweet_timestamp"] < '2020-02-11 06:00:00')]
t12 = df[(df["tweet_timestamp"] >= '2020-02-11 06:00:00') & (df["tweet_timestamp"] < '2020-02-11 18:00:00')]
t13 = df[(df["tweet_timestamp"] >= '2020-02-11 18:00:00') & (df["tweet_timestamp"] < '2020-02-12 06:00:00')]
t14 = df[(df["tweet_timestamp"] >= '2020-02-12 06:00:00') & (df["tweet_timestamp"] < '2020-02-12 18:00:00')]

# Each time period is 12 hrs
# Engagement frequency of user is from time periods t1 to t11 during training
# Recent history of user is considered from time periods t8 to t11 training 
# Training will be carrid out on time period t12
# Validation carried out on time period t13
# Testing will be carried out on time period t14


In [9]:
#Engagement frequency for all available timeperiod
eng_frequency = df[df["tweet_timestamp"] < '2020-02-12 6:00:00']
eng_frequency['retweet'] = np.where(pd.notnull(eng_frequency['retweet_timestamp']), 1, 0)
columns = ['tweet_timestamp', 'tweet_id', 'reply_timestamp', "retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
eng_frequency.drop(columns, axis=1,inplace=True)
engagement_history = eng_frequency.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
input_engagement_history = engagement_history.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
input_engagement_history.fillna(0,inplace = True)
history_frequency = pd.DataFrame(input_engagement_history.to_records())

In [10]:
##Engagement frequency for testing time
eng_frequency_test = df[df["tweet_timestamp"] < '2020-02-12 6:00:00']
eng_frequency_test['retweet'] = np.where(pd.notnull(eng_frequency_test['retweet_timestamp']), 1, 0)
columns = ['tweet_timestamp', 'tweet_id', 'reply_timestamp', "retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
eng_frequency_test.drop(columns, axis=1,inplace=True)
engagement_history_test = eng_frequency_test.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
input_engagement_history_test = engagement_history_test.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
input_engagement_history_test.fillna(0,inplace = True)
history_frequency_test = pd.DataFrame(input_engagement_history_test.to_records())
left_out_rows_xt = history_frequency[~history_frequency['engaging_user_id'].isin(history_frequency_test['engaging_user_id'])]
for col in left_out_rows_xt.columns:
    if col != 'engaging_user_id':
        left_out_rows_xt[col].values[:] = 0
final_history_xt = history_frequency_test.append(left_out_rows_xt)
final_history_xt = final_history_xt.sort_values('engaging_user_id')
history_frequency_test = final_history_xt.reset_index(drop=True)



In [11]:
#Engagement frequency for training time
eng_frequency_train = df[df["tweet_timestamp"] < '2020-02-11 6:00:00']
eng_frequency_train['retweet'] = np.where(pd.notnull(eng_frequency_train['retweet_timestamp']), 1, 0)
columns = ['tweet_timestamp', 'tweet_id', 'reply_timestamp', "retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
eng_frequency_train.drop(columns, axis=1,inplace=True)
engagement_history_train = eng_frequency_train.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
input_engagement_history_train = engagement_history_train.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
input_engagement_history_train.fillna(0,inplace = True)
history_frequency_train = pd.DataFrame(input_engagement_history_train.to_records())
left_out_rows_xt = history_frequency[~history_frequency['engaging_user_id'].isin(history_frequency_train['engaging_user_id'])]
for col in left_out_rows_xt.columns:
    if col != 'engaging_user_id':
        left_out_rows_xt[col].values[:] = 0
final_history_xt = history_frequency_train.append(left_out_rows_xt)
final_history_xt = final_history_xt.sort_values('engaging_user_id')
history_frequency_train = final_history_xt.reset_index(drop=True)

In [12]:


columns = ['reply_timestamp','tweet_timestamp', 'tweet_id',"retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]


In [13]:
def final_history(z):
    xt = z
    columns = ['reply_timestamp','tweet_timestamp', 'tweet_id',"retweet_timestamp","retweet_with_comment_timestamp","like_timestamp"]
    xt['retweet'] = np.where(pd.notnull(xt['retweet_timestamp']), 1, 0)
    xt.drop(columns, axis=1)
    x_t = xt.groupby(['engaging_user_id', 'text_tokens'])[['retweet']].agg('sum')
    xt_history = x_t.pivot_table(index='engaging_user_id', columns='text_tokens', values='retweet')
    xt_history.fillna(0,inplace = True)
    history_xt = pd.DataFrame(xt_history.to_records())
    for col in history_xt.columns:
      if col != 'engaging_user_id':
        history_xt.loc[history_xt[col] > 1, col] = 1

    left_out_rows_xt = history_frequency[~history_frequency['engaging_user_id'].isin(history_xt['engaging_user_id'])]
    for col in left_out_rows_xt.columns:
        if col != 'engaging_user_id':
            left_out_rows_xt[col].values[:] = 0
    final_history_xt = history_xt.append(left_out_rows_xt)
    final_history_xt = final_history_xt.sort_values('engaging_user_id')
    final_history_xt.reset_index(drop=True)
    return final_history_xt

In [14]:
#dataframes with engagement histories  
from functools import reduce
recent_history = reduce(lambda x,y: pd.merge(x,y, on='engaging_user_id', how='outer'), [final_history(t8), final_history(t9), final_history(t10), final_history(t11)])
recent_history_test = reduce(lambda x,y: pd.merge(x,y, on='engaging_user_id', how='outer'), [final_history(t10), final_history(t11), final_history(t12), final_history(t13)])

In [15]:
history_frequency = history_frequency.sort_values('engaging_user_id')
history_frequency_train = history_frequency_train.sort_values('engaging_user_id')

In [16]:
#topic recommendations generation for training and testing.

time = df[(df["tweet_timestamp"] >= '2020-02-12 06:00:00') & (df["tweet_timestamp"] < '2020-02-12 18:00:00')]
#time = df[(df["tweet_timestamp"] >= '2020-02-10 06:00:00') & (df["tweet_timestamp"] < '2020-02-10 18:00:00')]
left = time[time['retweet_timestamp'] >= 0]
time['retweet_timestamp'].fillna(0,inplace=True)

left['retweet_timestamp'] = left["retweet_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x))
time = pd.concat([time,left]).drop_duplicates(keep=False)
time['retweet_timestamp'] = time['tweet_timestamp']
final = pd.concat([time,left])

final = final.sort_values(['engaging_user_id', 'retweet_timestamp'], ascending=[True, False])
final = final.reset_index(drop=True)

columns = ['tweet_id',"reply_timestamp","retweet_with_comment_timestamp","like_timestamp"]
final.drop(columns, axis=1,inplace=True)


time_1 = df[(df["tweet_timestamp"] >= '2020-02-11 6:00:00') & (df["tweet_timestamp"] < '2020-02-11 18:00:00')]
left_1 = time_1[time_1['retweet_timestamp'] >= 0]
time_1['retweet_timestamp'].fillna(0,inplace=True)

left_1['retweet_timestamp'] = left_1["retweet_timestamp"].apply(lambda x: datetime.datetime.fromtimestamp(x))
time_1 = pd.concat([time_1,left_1]).drop_duplicates(keep=False)
time_1['retweet_timestamp'] = time_1['tweet_timestamp']
initial = pd.concat([time_1,left_1])

initial = initial.sort_values(['engaging_user_id', 'retweet_timestamp'], ascending=[True, False])
initial = initial.reset_index(drop=True)

columns = ['tweet_id',"reply_timestamp","retweet_with_comment_timestamp","like_timestamp"]
initial.drop(columns, axis=1,inplace=True)

In [17]:
# We consider the positive examples for topic recommendations from all the tweets posted during the active states and vice versa. 
# An active state is defined as a period between when a tweet is published and engaged


#Function action_recommend for determining the active states
def active_recommend(x):
    g = 3
    for index, row in x.iterrows():
      p = 0  
      if g != 10 :
        a = row['engaging_user_id']
        b = row['tweet_timestamp']
        c = row['retweet_timestamp']
        p = 0
        if b != c :
          g = 10
      if (row['tweet_timestamp'] != row['retweet_timestamp']):
          p = 1
          d = row['engaging_user_id']
          e = row['tweet_timestamp']
          f = row['retweet_timestamp']
      elif row['tweet_timestamp'] == row['retweet_timestamp']:
          if row['engaging_user_id'] == a:
            p = 0
          elif row['engaging_user_id'] == d:
            if (row['tweet_timestamp'] >= e) & (row['tweet_timestamp'] <= f):
              p = 1
            else:
              p = 0
      x.loc[index,'recommend'] = p
    return x
final = active_recommend(final)    # Test Recommendations 
inital = active_recommend(initial) # Training Recommendations

In [18]:
 column = ['tweet_timestamp','retweet_timestamp']
 final.drop(column, axis=1,inplace=True)
 initial.drop(column, axis=1,inplace=True)

In [19]:

final_1 = final.groupby(['engaging_user_id', 'text_tokens'])[['recommend']].agg('sum')
final_2 = final_1.pivot_table(index='engaging_user_id', columns='text_tokens', values='recommend')
final_2.fillna(0,inplace = True)
final_3 = pd.DataFrame(final_2.to_records())

for col in final_3.columns:
  if col != 'engaging_user_id':
    final_3.loc[final_3[col] > 1, col] = 1
    
    
left_out_rows_f = history_frequency[~history_frequency['engaging_user_id'].isin(final_3['engaging_user_id'])]
for col in left_out_rows_f.columns:
    if col != 'engaging_user_id':
        left_out_rows_f[col].values[:] = 0
final_4 = final_3.append(left_out_rows_f)
recommend_test = final_4.sort_values('engaging_user_id')
recommend_test.reset_index(drop=True, inplace=True)  #Dataframe with topic recommendation during testing

In [20]:
initial_1 = initial.groupby(['engaging_user_id', 'text_tokens'])[['recommend']].agg('sum')
initial_2 = initial_1.pivot_table(index='engaging_user_id', columns='text_tokens', values='recommend')
initial_2.fillna(0,inplace = True)
initial_3 = pd.DataFrame(final_2.to_records())

for col in initial_3.columns:
  if col != 'engaging_user_id':
    initial_3.loc[final_3[col] > 1, col] = 1
    
left_out_rows_g = history_frequency[~history_frequency['engaging_user_id'].isin(final_3['engaging_user_id'])]
for col in left_out_rows_g.columns:
    if col != 'engaging_user_id':
        left_out_rows_f[col].values[:] = 0
initial_4 = initial_3.append(left_out_rows_f)
recommend = initial_4.sort_values('engaging_user_id')
recommend.reset_index(drop=True, inplace=True)      #Dataframe with topic recommendation during training
    

In [21]:
X_recent_2 = recent_history.drop('engaging_user_id', axis=1).values
X_recent_1 = X_recent_2.reshape(-1,50,4)

X_recent_2_test = recent_history_test.drop('engaging_user_id', axis=1).values
X_recent_1_test = X_recent_2_test.reshape(-1,50,4)



X_freq_1 = history_frequency_test.drop('engaging_user_id', axis=1).values
X_freq_1_train = history_frequency_train.drop('engaging_user_id', axis=1).values


X_recommend_1 = recommend.drop('engaging_user_id', axis=1).values
X_recommend_t = recommend_test.drop('engaging_user_id', axis=1).values

y_1 = final_history(t12).drop('engaging_user_id', axis=1).values

y_t = final_history(t14).drop('engaging_user_id', axis=1).values

#INPUTS



In [22]:
X_recent = torch.FloatTensor(X_recent_1)              #Recent Engagement History tensor input for training
X_freq = torch.FloatTensor(X_freq_1_train)            #Engagement Frequency tensor input for training
X_recommend = torch.FloatTensor(X_recommend_1)        #Topic Recommendation tensor input for training
y = torch.FloatTensor(y_1)                            #Engagement output tensor input for training


X_recent_test = torch.FloatTensor(X_recent_1_test)    #Recent Engagement History tensor input for testing
X_freq_test = torch.FloatTensor(X_freq_1)             #Engagement Frequency tensor input for testing
X_recommend_test = torch.FloatTensor(X_recommend_t)   #Topic Recommendation tensor input for testing
Y_test = torch.FloatTensor(y_t)                       #Engagement output tensor input for testing

In [23]:
X_freq = F.normalize(X_freq, p=2, dim=1)              #Normalized Engagement Frequency tensor input for training
X_freq_test = F.normalize(X_freq_test, p=2, dim=1)    #Normalized Engagement Frequency tensor input for testing

In [60]:
filters = 20
L = 10
class DNN_f(nn.Module):
    def __init__(self):
        super().__init__()
        #frequency input
        self.fc1 = nn.Linear(in_features=50, out_features=L)

        #history input 32 * 50 * 4
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = filters, kernel_size = (1,4) ,stride = 1)
        self.leaky = nn.LeakyReLU()
        self.conv2 = nn.Conv2d(in_channels = filters, out_channels = filters, kernel_size = (50-L+1,1) ,stride = 1)
        self.conv3 = nn.ConvTranspose2d(in_channels = filters, out_channels = filters, kernel_size =(50-L+1,1), stride =1)


        #recommend input
        self.fc3 = nn.Linear(in_features=50, out_features=L)
        
        #final linear 
        self.conv4 = nn.Conv2d(in_channels = 1, out_channels=1, kernel_size=(1, 5+2*(filters)), stride = 1) 
        

 
    def forward(self, x, y, z, a):
        x1 = self.fc1(x)      ### EH
        x2 = F.linear(x1, self.fc1.weight.t())  #### EH BAR

        
        y_ = y.view(-1,1,50,4)
        y1 = self.conv1(y_)
        y1 = self.leaky(y1)
        y2 = y1.view(-1,50,filters)   ####   ET       
        y1 = self.conv2(y1)
        y1 = self.conv3(y1)
        #y1 = self.leaky(y1)
        y3 = y1.view(-1,50,filters)   ####   ETBAR
        
        z1 = self.fc3(z)    ####  DT
        z2 = F.linear(z1, self.fc3.weight.t())    #### DTBAR

        
        w = torch.stack((x, x2,z, z2), dim=2)
        v = torch.cat((y2,y3),dim =2)
        u = torch.cat((w,v),dim=2)
        a = a.view(-1,50,1)
        r = torch.cat((a,u),dim = 2)
        
        r = r.view(-1,1,50,5+2*(filters))
        t = self.conv4(r)
        s = t.view(-1,50)

        return s

In [61]:
torch.ones([32, 50], dtype=torch.float64).shape

torch.Size([32, 50])

In [26]:
model = DNN_f()
model = model.to(device)

In [27]:
import numpy as np
import math
from sklearn.metrics import log_loss
# custom function
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# define vectorized sigmoid
sigmoid_v = np.vectorize(sigmoid)


In [28]:
optimizer = Adam(model.parameters(), lr=1e-5)
loss_func = nn.BCEWithLogitsLoss(reduction = 'mean').cuda()

BATCH_SIZE = 32
train_loss = 0
losses = []
steps = []
step = 0
count = 0
EPOCHS = 50
trained = []
tested = []

In [29]:
X_indicator = torch.ones([107017, 50])

In [30]:

import time
start_time = time.time()
train_dataset = TensorDataset(X_freq, X_recent, X_recommend, y, X_indicator)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
print ("--- %s seconds ---" % (time.time() - start_time))


for epoch_num in range(EPOCHS):
    model.train()
    for step_num, batch_data in enumerate(train_dataloader):
        freq, recent, recommend, labels, indicator = tuple(t.to(device)for t in batch_data)
        probas = model(freq, recent, recommend, indicator)
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()

        model.zero_grad()
        batch_loss.backward()


        #clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()

        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("{0}/{1} loss: {2} ".format(step_num, len(y)/BATCH_SIZE, train_loss / (count + 1)))
        print ("--- %s seconds ---" % (time.time() - start_time))
        losses.append(batch_loss.item())
        steps.append(step)
        step += 1
        count += 1
#     lg_train = model(X_freq.to(device), X_recent.to(device), X_recommend.to(device), X_indicator.to(device)).to('cpu')
#     numpy_lg_train = lg_train.detach().numpy()
#     prob_tr = sigmoid_v(numpy_lg_train)
#     trained.append(log_loss(y.numpy(),prob_tr))
#     lg_test = model(X_freq_test.to(device), X_recent_test.to(device), X_recommend_test.to(device), X_indicator.to(device)).to('cpu')
#     numpy_lg_test = lg_test.detach().numpy()
#     prob_tt = sigmoid_v(numpy_lg_test)
#     tested.append(log_loss(Y_test.numpy(),prob_tt))


Epoch:  50
3344/3344.28125 loss: 0.042608323653697724 
--- 803.8426699638367 seconds ---


In [62]:
model = torch.load("model_dnn_march30_12743.pkl")
model = model.to(device)

In [63]:
model.eval()

with torch.no_grad():

    logits_train = model(X_freq.to(device), X_recent.to(device), X_recommend.to(device), X_indicator.to(device)).to('cpu')
    logits_test = model(X_freq_test.to(device), X_recent_test.to(device), X_recommend_test.to(device), X_indicator.to(device)).to('cpu')
    numpy_logits_train = logits_train.detach().numpy()
    numpy_logits_test = logits_test.detach().numpy()
    
        

In [64]:
#Converting logits to probabilities
numpy_probas_test = sigmoid_v(numpy_logits_test)
numpy_probas_train = sigmoid_v(numpy_logits_train)

In [65]:
from sklearn.metrics import log_loss
log_loss(Y_test.numpy(),numpy_probas_test)

1.095926663173644

In [66]:
thresh = []
roc = []
for i in range(50):    
    import sklearn.metrics as metrics
    fpr, tpr, threshold = metrics.roc_curve(Y_test[:,i].numpy(), numpy_probas_test[:,i])
    roc_auc = metrics.auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    roc.append(roc_auc)
    thresh.append(optimal_threshold)

In [67]:
sum(roc)/50

0.8972899944806862

In [36]:
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

for i in range(50):
    numpy_probas_test[:,i] = to_labels(numpy_probas_test[:,i],thresh[i])

In [37]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(Y_test.numpy(), numpy_probas_test)

array([[[100736,   5483],
        [    98,    700]],

       [[ 74843,  31941],
        [    19,    214]],

       [[ 83876,  22982],
        [    34,    125]],

       [[ 97838,   8577],
        [    93,    509]],

       [[ 81226,  23723],
        [   465,   1603]],

       [[ 85239,  20306],
        [   246,   1226]],

       [[ 91839,  15140],
        [    10,     28]],

       [[ 80503,  23987],
        [   332,   2195]],

       [[ 77521,  28976],
        [    95,    425]],

       [[ 81851,  14214],
        [   759,  10193]],

       [[ 94977,  11861],
        [    57,    122]],

       [[ 87597,  17897],
        [   171,   1352]],

       [[ 85873,  21107],
        [    12,     25]],

       [[ 90741,  15067],
        [   329,    880]],

       [[ 98217,   8150],
        [   107,    543]],

       [[ 79739,  27152],
        [    33,     93]],

       [[ 71264,  35702],
        [     8,     43]],

       [[ 85034,  18729],
        [   355,   2899]],

       [[ 85758,  20612],
  

# Binary Logits

In [38]:
#Converting engagement frequencies from tensor to numpy array

X_freq_1 = X_freq.numpy()
X_freq_1_test = X_freq_test.numpy()


In [39]:
empty_train_p = []
empty_test_p = []
for i in range(50):
    recent_lm_train = X_recent_2[:,[i,i+50,i+100,i+150]]     #Engagement history by topic as input for training
    freq_lm_train = X_freq_1[:,i]                            #Engagement frequncy by topic as input for training
    recommend_lm_train = X_recommend_1[:,i]                  #Recommendations by topic as input for training
    
    recent_lm_test = X_recent_2_test[:,[i,i+50,i+100,i+150]] #Engagement history by topic as input for testing
    freq_lm_test = X_freq_1_test[:,i]                        #Engagement frequncy by topic as input for testing
    recommend_lm_test = X_recommend_t[:,i]                   #Recommendations by topic as input for testing


    y_lm_train = y_1[:,i]                                    #Engagement labels

    X = np.column_stack([recent_lm_train,freq_lm_train,recommend_lm_train])  #Concatenated input features for training

    #LR = LogisticRegression(class_weight="balanced")
    LR = LogisticRegression()

    LR.fit(X,y_lm_train)
    
    X_test = np.column_stack([recent_lm_test,freq_lm_test,recommend_lm_test]) #Concatenated input features for testing
    
    empty_train_p.append((LR.predict_proba(X)[:,1]))
       
    empty_test_p.append((LR.predict_proba(X_test)[:,1]))
      

In [40]:
prediction_lr_train_p = np.transpose(np.array(empty_train_p))  #Predictions for training data
prediction_lr_test_p = np.transpose(np.array(empty_test_p))    #Predictions for test data

In [41]:
from sklearn.metrics import log_loss
log_loss(y_t,prediction_lr_test_p)

1.1467783394645972

In [42]:
thresh_lr = []
roc_lr = []
for i in range(50):    
    import sklearn.metrics as metrics
    fpr, tpr, threshold = metrics.roc_curve(Y_test[:,i].numpy(), prediction_lr_test_p[:,i])
    roc_auc = metrics.auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    thresh_lr.append(optimal_threshold)
    roc_lr.append(roc_auc)
    

In [43]:
sum(roc_lr)/50  #Average AUC-ROC score 

0.8061488027555269

In [44]:
label_lr_test_p = np.empty([107017, 50])
for i in range(50):
    label_lr_test_p[:,i] = to_labels(prediction_lr_test_p[:,i],thresh_lr[i])

In [45]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(Y_test.numpy(), label_lr_test_p)


array([[[102417,   3802],
        [     0,    798]],

       [[105775,   1009],
        [     0,    233]],

       [[105828,   1030],
        [     0,    159]],

       [[104243,   2172],
        [     0,    602]],

       [[ 94174,  10775],
        [     0,   2068]],

       [[102158,   3387],
        [     0,   1472]],

       [[106677,    302],
        [     0,     38]],

       [[ 92050,  12440],
        [     0,   2527]],

       [[ 98068,   8429],
        [     0,    520]],

       [[ 81224,  14841],
        [     0,  10952]],

       [[105311,   1527],
        [     0,    179]],

       [[100687,   4807],
        [     0,   1523]],

       [[106664,    316],
        [    36,      1]],

       [[ 99024,   6784],
        [     0,   1209]],

       [[ 99346,   7021],
        [     0,    650]],

       [[105812,   1079],
        [     0,    126]],

       [[106966,      0],
        [    51,      0]],

       [[ 91373,  12390],
        [     0,   3254]],

       [[100715,   5655],
  

# Light GBM model

In [46]:
empty_train_1p = []
empty_test_1p = []


for i in range(50):
    recent_lm_train = X_recent_2[:,[i,i+50,i+100,i+150]]
    freq_lm_train = X_freq_1[:,i]
    recommend_lm_train = X_recommend_1[:,i]
    
    recent_lm_test = X_recent_2_test[:,[i,i+50,i+100,i+150]]
    freq_lm_test = X_freq_1_test[:,i]
    recommend_lm_test = X_recommend_t[:,i]


    y_lm_train = y_1[:,i]
    

    X = np.column_stack([recent_lm_train,freq_lm_train,recommend_lm_train])

    LGBM = LGBMClassifier()

    LGBM.fit(X,y_lm_train)
    
    X_test = np.column_stack([recent_lm_test,freq_lm_test,recommend_lm_test])
    
    empty_train_1p.append((LGBM.predict_proba(X)[:,1]))

    empty_test_1p.append((LGBM.predict_proba(X_test)[:,1]))
        


In [47]:
prediction_lgbm_train_p = np.transpose(np.array(empty_train_1p))
prediction_lgbm_test_p = np.transpose(np.array(empty_test_1p))

In [48]:
from sklearn.metrics import log_loss
log_loss(Y_test.numpy(),prediction_lgbm_test_p)

1.1886555611206588

In [49]:
thresh_lgbm = []
roc_lgbm = []
for i in range(50):    
    import sklearn.metrics as metrics
    # calculate the fpr and tpr for all thresholds of the classification
    #probs = model.predict_proba(X_test)
    #preds = probs[:,1]
    fpr, tpr, threshold = metrics.roc_curve(Y_test[:,i].numpy(), prediction_lgbm_test_p[:,i])
    roc_auc = metrics.auc(fpr, tpr)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = threshold[optimal_idx]
    thresh_lgbm.append(optimal_threshold)
    roc_lgbm.append(roc_auc)
    


In [50]:
sum(roc_lgbm)/50

0.5451390615155378

In [51]:
label_lgbm_test_p = np.empty([107017, 50])
for i in range(50):
    label_lgbm_test_p[:,i] = to_labels(prediction_lgbm_test_p[:,i],thresh_lgbm[i])

In [52]:
multilabel_confusion_matrix(Y_test.numpy(), label_lgbm_test_p)

array([[[102684,   3535],
        [     2,    796]],

       [[105947,    837],
        [   223,     10]],

       [[106240,    618],
        [   156,      3]],

       [[104074,   2341],
        [    11,    591]],

       [[ 90466,  14483],
        [    12,   2056]],

       [[101831,   3714],
        [    19,   1453]],

       [[   272, 106707],
        [     0,     38]],

       [[ 91043,  13447],
        [    11,   2516]],

       [[ 97113,   9384],
        [   288,    232]],

       [[ 82736,  13329],
        [    33,  10919]],

       [[106828,     10],
        [   146,     33]],

       [[ 96988,   8506],
        [     4,   1519]],

       [[   244, 106736],
        [     0,     37]],

       [[ 98242,   7566],
        [     3,   1206]],

       [[100455,   5912],
        [     5,    645]],

       [[106783,    108],
        [    26,    100]],

       [[   211, 106755],
        [     0,     51]],

       [[ 90293,  13470],
        [    11,   3243]],

       [[ 98647,   7723],
  

# Exhaustive search for Optimization

In [53]:
x = pd.DataFrame(X_recommend_t)
x.sum(axis=1).value_counts()
x['Total'] = x.sum(axis=1)
y=x[x['Total'] >= 6] #Considering only the users who are shown atleast six tweets
#del y['Total']

infinity = pd.DataFrame(X_freq_1_test)
histo = pd.DataFrame(X_recent_2_test)

inf_new=infinity.loc[y.index,:]
hist_new=histo.loc[y.index,:]

# Greedy Search

In [68]:
def greedy(freq, hist, y):
    empty =[]
    for user in range(y.shape[0]):
        s=set()
        for i,x in enumerate(y.iloc[user]):
            if x==1.0:
                s.add(i)
        output=[[0.0]*50]
        removed=set()
        l_=[]

        for j in range(5):
            ma=-float('inf')
            for i in s:
                l_.append(i)

                output[0][i]=1.0
                inf_new_ = torch.FloatTensor(freq.iloc[user].values).reshape(1,50)
                hist_new_ = torch.FloatTensor(hist.iloc[user].values.reshape(-1,50,4))
                y_ = torch.FloatTensor(np.array(output[0])).reshape(1,50)
                indicator = torch.ones([50]).reshape(1,50)    
                logits_opt = model(inf_new_.to(device), hist_new_.to(device), y_.to(device), indicator.to(device))
                numpy_logits_opt = logits_opt.to('cpu').detach().numpy()
                numpy_probas_opt = sigmoid_v(numpy_logits_opt)
                sum_=sum(numpy_probas_opt[0][k] for k in l_)
                if sum_>ma:
                    result_l=l_[:]
                    numpy_logits_opt_max=numpy_logits_opt
                    numpy_probas_opt_max=numpy_probas_opt
                    ma=sum_
                    index=i
                l_.pop()
                output[0][i]=0.0

            output[0][index]=1.0
            l_.append(index)

            s.remove(index)
        empty.append([y.index[user],ma,result_l])
    n=pd.DataFrame(empty,columns=['USER','MAX_SUM','LIST_OF_INDEX'])
    return (n['MAX_SUM'].mean())
        

In [69]:
greedy(inf_new, hist_new, y)

1.677213976646597

# Uplift score obtained using combination of topics a given model

In [70]:
#prob : output probabilities for a given model
#y : input recommendations considering overall timeline
#freq: engagement frequency
#hist: engagement history

def uplift_score(prob, freq, hist, y):     
    mdl = pd.DataFrame(prob)
    mdl_y = mdl.loc[y.index]
    p = mdl_y*y.iloc[:,0:50].values
    s=pd.DataFrame(np.zeros((p.shape[0],p.shape[1]))).set_index(p.index)
    for ind in p.index:
        s.loc[ind,p.loc[ind].nlargest(5).index.values]=1
    s_=s.reset_index()
    s_.index = s_['index']
    del s_['index']
    inf_new_T = torch.FloatTensor(freq.values)
    hist_new_T = torch.FloatTensor(hist.values.reshape(-1,50,4))
    rec_T = torch.FloatTensor(s_.values)
    indicator_T = torch.ones([len(rec_T), 50])
    logits_opt = model(inf_new_T.to(device), hist_new_T.to(device), rec_T.to(device), indicator_T.to(device))
    numpy_logits_opt = logits_opt.to('cpu').detach().numpy()
    numpy_probas_opt = sigmoid_v(numpy_logits_opt)
    sp = pd.DataFrame(numpy_probas_opt, index = y.index)
    sc = sp*s_.values
    return(sc.sum(axis = 1).mean())

In [71]:
uplift_score(prediction_lr_test_p,inf_new, hist_new, y)

1.6448964056409812

In [72]:
uplift_score(prediction_lgbm_test_p,inf_new, hist_new, y)

1.6368377623062471